# Lambda Functions and Pivot Tables

Until now, we have not made any changes or modifications to the data. In this section, we will:
* Use lambda functions to create new and alter existing columns
* Use pandas pivot tables as an alternative to ```df.groupby()``` to summarise data

Let's first read all the files and create a ```master_df```. 

In [1]:
# Loading libraries and files
import numpy as np
import pandas as pd

market_df = pd.read_csv("market_fact.csv")
customer_df = pd.read_csv("cust_dimen.csv")
product_df = pd.read_csv("prod_dimen.csv")
shipping_df = pd.read_csv("shipping_dimen.csv")
orders_df = pd.read_csv("orders_dimen.csv")

# Merging the dataframes to create a master_df
df_1 = pd.merge(market_df, customer_df, how='inner', on='Cust_id')
df_2 = pd.merge(df_1, product_df, how='inner', on='Prod_id')
df_3 = pd.merge(df_2, shipping_df, how='inner', on='Ship_id')
master_df = pd.merge(df_3, orders_df, how='inner', on='Ord_id')

master_df.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,...,Region,Customer_Segment,Product_Category,Product_Sub_Category,Order_ID_x,Ship_Mode,Ship_Date,Order_ID_y,Order_Date,Order_Priority
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.51,3.60,0.56,...,WEST,CORPORATE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",36262,REGULAR AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED
1,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59,...,WEST,CORPORATE,TECHNOLOGY,TELEPHONES AND COMMUNICATION,36262,EXPRESS AIR,27-07-2010,36262,27-07-2010,NOT SPECIFIED
2,Ord_5446,Prod_6,SHP_7608,Cust_1818,164.02,0.03,23,-47.64,6.15,0.37,...,WEST,CORPORATE,OFFICE SUPPLIES,PAPER,36262,EXPRESS AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED
3,Ord_2978,Prod_16,SHP_4112,Cust_1088,305.05,0.04,27,23.12,3.37,0.57,...,ONTARIO,HOME OFFICE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",37863,REGULAR AIR,26-02-2011,37863,24-02-2011,HIGH
4,Ord_5484,Prod_16,SHP_7663,Cust_1820,322.82,0.05,35,-17.58,3.98,0.56,...,WEST,CONSUMER,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",53026,REGULAR AIR,03-03-2012,53026,26-02-2012,LOW


### Lambda Functions

Say you want to create a new column indicating whether a given order was profitable or not (1/0). 

You need to apply a function which returns 1 if Profit > 0, else 0. This can be easily done using the ```apply()``` method on a column of the dataframe. 

In [9]:
# Create a function to be applied
def is_positive(x):
    return x > 0

# Create a new column
master_df['isProfitable'] = master_df['Profit'].apply(is_positive)
master_df.head()
master_df.drop(['is_profitable'],inplace=False,axis=1)

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,...,Customer_Segment,Product_Category,Product_Sub_Category,Order_ID_x,Ship_Mode,Ship_Date,Order_ID_y,Order_Date,Order_Priority,isProfitable
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.51,3.60,0.56,...,CORPORATE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",36262,REGULAR AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False
1,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59,...,CORPORATE,TECHNOLOGY,TELEPHONES AND COMMUNICATION,36262,EXPRESS AIR,27-07-2010,36262,27-07-2010,NOT SPECIFIED,True
2,Ord_5446,Prod_6,SHP_7608,Cust_1818,164.02,0.03,23,-47.64,6.15,0.37,...,CORPORATE,OFFICE SUPPLIES,PAPER,36262,EXPRESS AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False
3,Ord_2978,Prod_16,SHP_4112,Cust_1088,305.05,0.04,27,23.12,3.37,0.57,...,HOME OFFICE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",37863,REGULAR AIR,26-02-2011,37863,24-02-2011,HIGH,True
4,Ord_5484,Prod_16,SHP_7663,Cust_1820,322.82,0.05,35,-17.58,3.98,0.56,...,CONSUMER,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",53026,REGULAR AIR,03-03-2012,53026,26-02-2012,LOW,False
5,Ord_3730,Prod_16,SHP_5175,Cust_1314,459.08,0.04,34,61.57,3.14,0.60,...,CORPORATE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",36992,EXPRESS AIR,09-12-2009,36992,07-12-2009,MEDIUM,True
6,Ord_4143,Prod_16,SHP_5771,Cust_1417,207.21,0.06,24,-78.64,6.14,0.59,...,HOME OFFICE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",13920,REGULAR AIR,10-10-2009,13920,09-10-2009,CRITICAL,False
7,Ord_4143,Prod_6,SHP_5770,Cust_1417,901.32,0.10,25,338.01,5.08,0.38,...,HOME OFFICE,OFFICE SUPPLIES,PAPER,13920,EXPRESS AIR,11-10-2009,13920,09-10-2009,CRITICAL,True
8,Ord_4796,Prod_16,SHP_6686,Cust_1659,95.09,0.09,9,-13.53,3.37,0.57,...,SMALL BUSINESS,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",55554,REGULAR AIR,17-11-2009,55554,16-11-2009,CRITICAL,False
9,Ord_4796,Prod_6,SHP_6686,Cust_1659,122.09,0.04,6,-15.20,9.54,0.37,...,SMALL BUSINESS,OFFICE SUPPLIES,PAPER,55554,REGULAR AIR,17-11-2009,55554,16-11-2009,CRITICAL,False


The same can be done in just one line of code using lambda functions. 

In [10]:
# Create a new column using a lambda function
master_df['is_profitable'] = master_df['Profit'].apply(lambda x: x > 1)
master_df.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,...,Product_Category,Product_Sub_Category,Order_ID_x,Ship_Mode,Ship_Date,Order_ID_y,Order_Date,Order_Priority,isProfitable,is_profitable
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.51,3.60,0.56,...,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",36262,REGULAR AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False,False
1,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59,...,TECHNOLOGY,TELEPHONES AND COMMUNICATION,36262,EXPRESS AIR,27-07-2010,36262,27-07-2010,NOT SPECIFIED,True,True
2,Ord_5446,Prod_6,SHP_7608,Cust_1818,164.02,0.03,23,-47.64,6.15,0.37,...,OFFICE SUPPLIES,PAPER,36262,EXPRESS AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False,False
3,Ord_2978,Prod_16,SHP_4112,Cust_1088,305.05,0.04,27,23.12,3.37,0.57,...,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",37863,REGULAR AIR,26-02-2011,37863,24-02-2011,HIGH,True,True
4,Ord_5484,Prod_16,SHP_7663,Cust_1820,322.82,0.05,35,-17.58,3.98,0.56,...,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",53026,REGULAR AIR,03-03-2012,53026,26-02-2012,LOW,False,False


Now you can use the new column to compare the percentage of profitable orders across groups.

In [11]:
# Comparing percentage of profitable orders across customer segments
by_segment = master_df.groupby('Customer_Segment')
by_segment.is_profitable.mean()

Customer_Segment
CONSUMER          0.496058
CORPORATE         0.477893
HOME OFFICE       0.491142
SMALL BUSINESS    0.491474
Name: is_profitable, dtype: float64

In [17]:
# Comparing percentage of profitable orders across product categories
by_category = master_df.groupby('Product_Category')
by_category.is_profitable.mean()

Product_Category
FURNITURE          0.464037
OFFICE SUPPLIES    0.458134
TECHNOLOGY         0.571913
Name: is_profitable, dtype: float64

In FURNITURE, 46% orders are profitable, compared to 57% in TECHNOLOGY. 

In [21]:
# You can also use apply and lambda to alter existing columns
# E.g. you want to see Profit as one decimal place
# apply the round() function 
master_df['Profit'] = master_df['Profit'].apply(lambda x: round(x, 0))
master_df.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,...,Product_Category,Product_Sub_Category,Order_ID_x,Ship_Mode,Ship_Date,Order_ID_y,Order_Date,Order_Priority,isProfitable,is_profitable
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.0,3.60,0.56,...,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",36262,REGULAR AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False,False
1,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1149.0,2.50,0.59,...,TECHNOLOGY,TELEPHONES AND COMMUNICATION,36262,EXPRESS AIR,27-07-2010,36262,27-07-2010,NOT SPECIFIED,True,True
2,Ord_5446,Prod_6,SHP_7608,Cust_1818,164.02,0.03,23,-48.0,6.15,0.37,...,OFFICE SUPPLIES,PAPER,36262,EXPRESS AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False,False
3,Ord_2978,Prod_16,SHP_4112,Cust_1088,305.05,0.04,27,23.0,3.37,0.57,...,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",37863,REGULAR AIR,26-02-2011,37863,24-02-2011,HIGH,True,True
4,Ord_5484,Prod_16,SHP_7663,Cust_1820,322.82,0.05,35,-18.0,3.98,0.56,...,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",53026,REGULAR AIR,03-03-2012,53026,26-02-2012,LOW,False,False


You sometimes need to create new columns using existing columns, for instance, say you want a column ```Profit / Order_Quantity```. 

In [29]:
# Creating a column Profit / Order_Quantity
master_df['profit_per_qty'] = master_df['Profit'] / master_df['Order_Quantity']
master_df.head()
master_df.loc[:,['profit_per_qty']]

,profit_per_qty
0,-1.304348
1,44.192308
2,-2.086957
3,0.851852
4,-0.514286
5,1.823529
6,-3.291667
7,13.520000
8,-1.555556
9,-2.500000


### Pivot Tables

You may want to use pandas pivot tables as an alternative to ```groupby()```. They provide Excel-like functionalities to create aggregate tables. 

In [30]:
# Read documentation
help(pd.DataFrame.pivot_table)

Help on function pivot_table in module pandas.core.frame:

pivot_table(self, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')
    Create a spreadsheet-style pivot table as a DataFrame. The levels in
    the pivot table will be stored in MultiIndex objects (hierarchical
    indexes) on the index and columns of the result DataFrame
    
    Parameters
    ----------
    values : column to aggregate, optional
    index : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same length as the data. The
        list can contain any of the other types (except list).
        Keys to group by on the pivot table index.  If an array is passed,
        it is being used as the same manner as column values.
    columns : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same length as the data. The
        list can contain any of the other types

The general syntax is ```pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', ...)```.
* ```data``` is a dataframe
* ```values``` contains the column to aggregate
* ```index``` is the row in the pivot table
* ```columns``` contains the columns you want in the pivot table
* ```aggfunc``` is the aggregate function

Let's see some examples.

In [38]:
# E.g. Compare average Sales across customer segments
master_df.pivot_table(values = 'Sales', index = 'Customer_Segment', aggfunc = 'mean')


,Sales
Customer_Segment,
CONSUMER,1857.859965
CORPORATE,1787.680389
HOME OFFICE,1754.312931
SMALL BUSINESS,1698.124841


In [10]:
# E.g. compare total number of profitable orders across regions
# Note that since is_profitable is 1/0, we can directly compute the sum
master_df.pivot_table(values = 'is_profitable', index = 'Region', aggfunc = 'sum')

,is_profitable
Region,
ATLANTIC,544.0
NORTHWEST TERRITORIES,194.0
NUNAVUT,38.0
ONTARIO,916.0
PRARIE,852.0
QUEBEC,360.0
WEST,969.0
YUKON,262.0


In [11]:
# Grouping by both rows and columns
# Compare the total profit across product categories and customer segments
# Since there are two categorical variables, we use both rows (index) and columns
master_df.pivot_table(values = 'Profit', 
                      index = 'Product_Category', 
                      columns = 'Customer_Segment', 
                      aggfunc = 'sum')

Customer_Segment,CONSUMER,CORPORATE,HOME OFFICE,SMALL BUSINESS
Product_Category,,,,
FURNITURE,42728.5,22008.3,23978.6,28717.5
OFFICE SUPPLIES,88532.4,203038.8,121145.6,105306.8
TECHNOLOGY,156700.1,374701.1,173230.6,181684.1


You don't necessarily need to specify all four arguments, since ```pivot_table()``` has some smart defaults. For instance, if you just provide ```columns```, it will compute the **mean of all the numeric columns** across each column. For e.g.:

In [12]:
# Computes the mean of all numeric columns across categories
# Notice that the means of Order_IDs are meaningless
master_df.pivot_table(columns = 'Product_Category')

Product_Category,FURNITURE,OFFICE SUPPLIES,TECHNOLOGY
Discount,0.049287,0.050230,0.048746
Order_ID_x,30128.711717,30128.122560,29464.891525
Order_ID_y,30128.711717,30128.122560,29464.891525
Order_Quantity,25.709977,25.656833,25.266344
Product_Base_Margin,0.598555,0.461270,0.556305
Profit,68.116531,112.369544,429.208668
Sales,3003.822820,814.048178,2897.941008
Shipping_Cost,30.883811,7.829829,8.954886
is_profitable,0.465197,0.466161,0.573366
profit_per_qty,-3.607020,1.736175,-52.274216
